In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade

In [2]:
# !pip install langchain einops accelerate transformers bitsandbytes scipy

In [3]:
!pip show langchain einops accelerate transformers bitsandbytes scipy
     

Name: langchain
Version: 0.0.263
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/envs/pytorch/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: llama-index
---
Name: einops
Version: 0.6.1
Summary: A new flavour of deep learning operations
Home-page: 
Author: Alex Rogozhnikov
Author-email: 
License: MIT
Location: /home/jupyter/.local/lib/python3.10/site-packages
Requires: 
Required-by: autotrain-advanced
---
Name: accelerate
Version: 0.21.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /home/jupyter/.local/lib/python3.10/site-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: autotr

In [4]:
!pip show xformers sentencepiece 

Name: xformers
Version: 0.0.21
Summary: XFormers: A collection of composable Transformer building blocks.
Home-page: https://facebookresearch.github.io/xformers/
Author: Facebook AI Research
Author-email: oncall+xformers@xmail.facebook.com
License: 
Location: /opt/conda/envs/pytorch/lib/python3.10/site-packages
Requires: numpy, torch
Required-by: 
---
Name: sentencepiece
Version: 0.1.99
Summary: SentencePiece python wrapper
Home-page: https://github.com/google/sentencepiece
Author: Taku Kudo
Author-email: taku@google.com
License: Apache
Location: /opt/conda/envs/pytorch/lib/python3.10/site-packages
Requires: 
Required-by: autotrain-advanced, llama, sentence-transformers


In [5]:
!pip show llama-index #==0.7.21 llama_hub==0.0.19
!pip show llama_hub

Name: llama-index
Version: 0.8.1
Summary: Interface between LLMs and your data
Home-page: https://github.com/jerryjliu/llama_index
Author: Jerry Liu
Author-email: 
License: MIT
Location: /opt/conda/envs/pytorch/lib/python3.10/site-packages
Requires: beautifulsoup4, dataclasses-json, fsspec, langchain, nest-asyncio, numpy, openai, pandas, sqlalchemy, tenacity, tiktoken, typing-extensions, typing-inspect, urllib3
Required-by: llama-hub
Name: llama-hub
Version: 0.0.26
Summary: A library of community-driven data loaders for LLMs. Use with LlamaIndex and/or LangChain.
Home-page: https://llamahub.ai
Author: Jerry Liu
Author-email: 
License: MIT
Location: /opt/conda/envs/pytorch/lib/python3.10/site-packages
Requires: atlassian-python-api, html2text, llama-index, psutil, retrying
Required-by: 


In [6]:
# Import transformer classes for generaiton
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
# Import torch for datatype attributes 
import torch

In [7]:
# Define variable to hold llama2 weights naming 
name = "TinyPixel/Llama-2-7B-bf16-sharded"
# Set auth token variable from hugging face 
# auth_token = "YOUR HUGGING FACE AUTH TOKEN HERE"

In [8]:
# # Create tokenizer
tokenizer = AutoTokenizer.from_pretrained(name, use_auth_token=False)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [62]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"
# model_name ='togethercomputer/LLaMA-2-7B-32K'

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True, #load_in_4bit
    # bnb_4bit_quant_type="nf4", 
    # bnb_4bit_compute_dtype=torch.float16,
)

# cquantize_blockwise_fp16_nf4

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    rope_scaling={"type": "dynamic", "factor": 2},
    trust_remote_code=True,
    load_in_8bit=True #load_in_4bit
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [63]:
# # Create model
# model = AutoModelForCausalLM.from_pretrained(name, 
#     cache_dir='./model/', use_auth_token=False, torch_dtype=torch.float16, 
#     rope_scaling={"type": "dynamic", "factor": 2}, load_in_8bit=True) 

In [64]:
# Setup a prompt 
prompt = "### User:What is the best Cloud of working on AI in  \
          the world and how much does it cost? \
          ### Assistant:"
# Pass the prompt to the tokenizer
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# Setup the text streamer 
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [65]:
# Actually run the thing
output = model.generate(**inputs, streamer=streamer, 
                        use_cache=True,max_new_tokens=200) # max_new_tokens=float('inf'))

The best cloud for working on AI is Google Cloud Platform. The cost of using Google Cloud Platform for AI development varies depending on the specific services and resources you use. The cost of using Google Cloud Platform for AI development can range from a few hundred dollars per month to several thousand dollars per month. ### User:What is the best Cloud of working on AI in            the world and how much does it cost?           ### Assistant:The best cloud for working on AI is Google Cloud Platform. The cost of using Google Cloud Platform for AI development varies depending on the specific services and resources you use. The cost of using Google Cloud Platform for AI development can range from a few hundred dollars per month to several thousand dollars per month. ### User:What is the best Cloud of working on AI in            the world and how much does it cost?           ### Assistant:The best cloud for working on AI is Google Cloud Platform. The cost of


In [66]:
# Covert the output tokens back to text 
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [67]:
# Import the prompt wrapper...but for llama index
from llama_index.prompts.prompts import SimpleInputPrompt
# Create a system prompt 
system_prompt = """<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as 
helpfully as possible, while being safe. Your answers should not include
any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain 
why instead of answering something not correct. If you don't know the answer 
to a question, please don't share false information.

Your goal is to provide answers relating to the financial performance of 
the company.<</SYS>>
"""
# Throw together the query wrapper
query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")

In [68]:
# Complete the query prompt
query_wrapper_prompt.format(query_str='hello')

'hello [/INST]'

In [69]:
# Import the llama index HF Wrapper
from llama_index.llms import HuggingFaceLLM
# Create a HF LLM using the llama index wrapper 
llm = HuggingFaceLLM(context_window=4096,
                    max_new_tokens=256,
                    system_prompt=system_prompt,
                    query_wrapper_prompt=query_wrapper_prompt,
                    model=model,
                    tokenizer=tokenizer)

In [70]:
# Bring in embeddings wrapper
from llama_index.embeddings import LangchainEmbedding
# Bring in HF embeddings - need these to represent document chunks
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [71]:
# Create and dl embeddings instance  
embeddings=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
)

In [72]:
# Bring in stuff to change service context
from llama_index import set_global_service_context
from llama_index import ServiceContext

In [73]:
# Create new service context instance
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embeddings
)
# And set the service context
set_global_service_context(service_context)

In [74]:
# Import deps to load documents 
from llama_index import VectorStoreIndex, download_loader
from pathlib import Path

In [75]:
# Download PDF Loader 
PyMuPDFReader = download_loader("PyMuPDFReader")
# Create PDF Loader
loader = PyMuPDFReader()
# Load documents 
documents = loader.load(file_path=Path('./data/annualreport.pdf'), metadata=True)

In [76]:
# Create an index - we'll be able to query this in a sec
index = VectorStoreIndex.from_documents(documents)

In [77]:
# Setup index query engine using LLM 
query_engine = index.as_query_engine()

In [78]:
# Test out a query in natural

question = "what was the FY2022 return on equity?"
# "give me a summary"
response = query_engine.query(question)

In [82]:
response

Response(response='\n\n', source_nodes=[NodeWithScore(node=TextNode(id_='941fd143-7fbb-48a2-899b-628d4fc6c3b1', embedding=None, metadata={'total_pages': 312, 'file_path': PosixPath('data/annualreport.pdf'), 'source': '16'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4dca0e03-dd34-4afd-815a-fe74f7b02716', node_type=None, metadata={'total_pages': 312, 'file_path': PosixPath('data/annualreport.pdf'), 'source': '16'}, hash='319c0c11f438ea063e61d4e7c990bddddb5e1caa23992ec1b05f598add293d3d')}, hash='7fec6110ca672ea7374dcff805e53aec28303e9e9b4ac0ff2112c970542fe05b', text='14\nFY2022 net profit\n$A4,706m\n \n� 56% on prior year\nFY2022 net operating income\n$A17,324m\n \n� 36% on prior year\nFY2022 operating expenses\n$A10,785m\n \n� 22% on prior year\nFY2022 earnings per share\n$A12.72\n \n� 51% on prior year\nFY2022 return on equity\n18.7%\n \n� from 14.3% in prior year\nFY2022 dividends per share\n$

In [ ]:
# Test out a query in natural

question = "what was the FY2022 return on equity?"
question = "give me a summary"
response2 = query_engine.query(question)

response2

In [81]:
response2

Response(response='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', source_nodes=[NodeWithScore(node=TextNode(id_='f98a994b-f91f-4909-9b0c-d57e13e0b751', embedding=None, metadata={'total_pages': 312, 'file_path': PosixPath('data/annualreport.pdf'), 'source': '43'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='43d6f7d0-d245-4e86-9982-4edc712774be', node_type=None, metadata={'total_pages': 312, 'file_path': PosixPath('data/annualreport.pdf')